In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn

RANDOM_SEED = 42

device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
df = pd.read_csv("./kidney-stone-dataset.csv")
df = df.drop(columns=["Unnamed: 0"])
X = df[["ph", "osmo", "urea", "calc", "gravity", "cond"]]
y = df["target"]  # or correct target column nameprint(df[:5])
X = df[["ph", "osmo", "urea", "calc", "gravity", "cond"]]
X = torch.from_numpy(np.array(X)).type(dtype=torch.float32)
y = torch.from_numpy(np.array(y)).type(dtype=torch.float32)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED
)


class Kidney_Stone(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(in_features=6, out_features=24)
        self.layer2 = nn.Linear(in_features=24, out_features=12)
        self.layer3 = nn.Linear(in_features=12, out_features=6)

    def forward(self, X):
        X = self.layer1(X)
        X = self.layer2(X)
        X = self.layer3(X)
        return X


torch.manual_seed(RANDOM_SEED)
model_7 = Kidney_Stone()

In [9]:

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(params=model_7.parameters(), lr=0.01)

In [27]:
print(X_train.size())
print(len(model_7.state_dict()))
epochs = 200
for epoch in range(epochs):
    model_7.train()
    y_logits = model_7(X_train)
    y_preds = torch.round(torch.sigmoid(y_logits))
    loss = loss_fn(y_preds, y_test)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(y_preds)
    model_7.eval()
    with torch.inference_mode():
        y_logits = model_7(X_test)
        test_preds = torch.round(torch.sigmoid(y_logits))
        test_loss = loss_fn(test_preds, y_test)

torch.Size([72, 6])
6


c:\Users\Haji Suleman\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:626: UserWarning: Using a target size (torch.Size([18])) that is different to the input size (torch.Size([72, 6])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (6) must match the size of tensor b (18) at non-singleton dimension 1